# <center> Ford vs Ferrari  
#### <center> Необходимо построить модель классификации изображений автомобилей по их фотографиям

---    
ИТОГИ РАБОТЫ НАД ПРОЕКТОМ:

+ Построить свой классификатор изображений.
+ Применить различные методы предобработки изображений.
+ Задействовать сразу несколько методов обучения (fine-tuning, transfer learning и так далее).
+ Научиться использовать предобученные модели для решения своих задач.
+ Найти и используете в работе State of the Art (SOTA)-модели.
---

<center> <img src="https://tribaltribune.org/wp-content/uploads/2020/01/960x0-1-1-900x515.jpg"/>

# Импорт библиотек и данных


In [2]:
# Проверим подключена ли видеокарта
!nvidia-smi

In [2]:
# Установим настраиваемый генератор данных изображений ImageDataAugmentor для аугментации данных
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint,CSVLogger, EarlyStopping
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import *
from ImageDataAugmentor.image_data_augmentor import *
import albumentations

from sklearn.model_selection import train_test_split, StratifiedKFold

from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.applications import EfficientNetB2
from keras.layers import BatchNormalization


import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

In [4]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS               = 5  # эпох на обучение (далее попробуем изменить количество эпох на обучение)
BATCH_SIZE           = 64 # уменьшаем batch если сеть большая, иначе не поместится в память на GPU
LR                   = 1e-4
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 224 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/sf-dl-car-classification/'
PATH = "../working/car/" # рабочая директория

In [5]:
# Фиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы
RANDOM_SEED=42 

# EDA (Разведывательный анализ данных)

In [6]:
# Ознакомимся с нашими данными
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [7]:
train_df.info()

In [8]:
train_df.Category.value_counts()

Как мы видим, данные сдержат 10 классов, а распределение классов достаточно равномерное.

In [9]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile(DATA_PATH+data_zip,"r") as z:
        z.extractall(PATH)
        
print(os.listdir(PATH))

In [10]:
print('Выведем случайный пример изображений:')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

В соответствии с представленными изображениями, можно сделать предположение о том что классы представляют из себя марки автомобилей

In [11]:
# Посмотрим на примеры картинок и их размеры, чтобы понимать, как их лучше обрабатывать и сжимать
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

# Аугментация данных

**Аугментация данных с помощью ImageDataGenerator:**

```python
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range = 5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=VAL_SPLIT, # set validation split
    horizontal_flip=False)

test_datagen = ImageDataGenerator(rescale=1. / 255)
```

**Аугментация данных с помощью albumentations:**

In [12]:
AUGMENTATIONS = albumentations.Compose([
    albumentations.transforms.HorizontalFlip(p=0.5),
#     albumentations.transforms.Flip(p=0.5),
    albumentations.transforms.FancyPCA (alpha=0.1, always_apply=False, p=0.5),
    albumentations.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
    albumentations.GaussianBlur(p=0.05),
#     albumentations.HueSaturationValue(p=0.8),
    albumentations.RGBShift(p=0.5),
    albumentations.FancyPCA(alpha=0.1, always_apply=False, p=0.8),
    albumentations.Resize(IMG_SIZE, IMG_SIZE)
])

In [13]:
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment=AUGMENTATIONS,
        validation_split=VAL_SPLIT)

# test_datagen = ImageDataAugmentor(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Генерация данных

In [14]:
# Завернём наши данные в генератор: 
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

# Построение модели

**Предобученная сверточная нейронная сеть Inception-ResNet-v2**:
```python
base_model = InceptionResNetV2(weights='imagenet',include_top=False, input_shape = input_shape)
```
> (Данная сеть не подходит, так как сеть является 164 слоями глубоко и может классифицировать изображения в 1 000 категорий объектов, таких как клавиатура, мышь, карандаш и многие животные)

**Предобученная нейронная сеть EfficientNetB2**:
```python
base_model = EfficientNetB2(include_top=False, weights='imagenet', input_tensor=None, 
    pooling=None, classes=1000, classifier_activation='softmax')
```

**Последовательная архитектура нейронной сети**
```python
base_model = Sequential()
base_model.add(Dense(224, input_shape=(input_shape)))
base_model.add(Activation('relu'))
base_model.add(Conv2D(224, (3, 3)))
base_model.add(Activation('relu'))
base_model.add(Dense(64))
base_model.add(Activation('relu'))
base_model.add(Dropout(0.5))
base_model.add(Dense(10))
base_model.add(Activation('sigmoid'))
base_model.add(BatchNormalization())
```

> Все вышеперечисленые модли не дал таких результатов как Xception. Соответственно далее будет использоваться Xception.
Посмотреть результаты работы других моделей можно пункте [Прочее](#Прочее)

In [15]:
# Предобученная сеть Xception:
base_model = Xception(weights='imagenet', include_top=False, input_shape = input_shape)

In [16]:
base_model.summary()

In [17]:
# Устанавливаем новую «голову» (head):
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(256, activation='relu')(x)
x = Dropout(0.25)(x)
# and a logistic layer -- let's say we have 10 classes
predictions = Dense(CLASS_NUM, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=LR), metrics=["accuracy"])

In [18]:
model.summary()

# Обучение модели

In [19]:
# Добавим ModelCheckpoint. Эта функция позволяет сохранять прогресс обучения модели, чтобы в нужный момент можно было его подгрузить
# и дообучить модель:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
callbacks_list = [checkpoint]

callbacks_stop = EarlyStopping(patience=5, restore_best_weights=True)
    
# Обратный вызов, который передает результаты эпохи в файл CSV.
# csv_logger = CSVLogger('training.log')
# csv_logger = [csv_logger]

In [20]:
# Обучаем
history = model.fit(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
        # callbacks=csv_logger
        # callbacks = callbacks_stop
)

#### → Рекомендация. Попробуйте применить transfer learning с fine-tuning.

In [21]:
# Сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model):
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [22]:
scores = model.evaluate(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

---
Изменение количества эпох на значение 10 с применением callback keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True) остановило обучение на 7 эпохе и не сильно повлияло на результат, зато процесс обучения увеличился.
> В итоге точность нашей модели составила 93 %. Даное зачение является выше чем baseline и учитывая, что классов десять, является хорошим результатом. Но можно больше.
---

In [23]:
# Посмотрим на графики обучения:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc)) 
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

## Предсказание на тестовых данных

In [24]:
test_sub_generator.samples

In [25]:
test_sub_generator.reset()
predictions = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [26]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

In [27]:
submission.head()

# Test Time Augmentation (TTA)

In [28]:
# tta_steps = 10
# predictions = []

# for i in tqdm(range(tta_steps)):
#     preds = model.predict_generator(train_datagen.flow(x_val, batch_size=bs, shuffle=False), steps = len(x_val)/bs)
#     predictions.append(preds)

# pred = np.mean(predictions, axis=0)

# np.mean(np.equal(np.argmax(y_val, axis=-1), np.argmax(pred, axis=-1)))

# <a name="Прочее"></a> Прочее

## **Предобученная сверточная нейронная сеть Inception-ResNet-v2**:

In [29]:
base_model_2 = InceptionResNetV2(weights='imagenet',include_top=False, input_shape = input_shape)

In [30]:
base_model_2.summary()

In [31]:
# Устанавливаем новую «голову» (head):
x = base_model_2.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(256, activation='relu')(x)
x = Dropout(0.25)(x)
# and a logistic layer -- let's say we have 10 classes
predictions = Dense(CLASS_NUM, activation='softmax')(x)

# this is the model we will train
model_2 = Model(inputs=base_model_2.input, outputs=predictions)
model_2.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=LR), metrics=["accuracy"])

In [32]:
model_2.summary()

In [33]:
# Обучение модели

# Добавим ModelCheckpoint. Эта функция позволяет сохранять прогресс обучения модели, чтобы в нужный момент можно было его подгрузить
# и дообучить модель:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
callbacks_list = [checkpoint]

callbacks_stop = EarlyStopping(patience=5, restore_best_weights=True)
    
# Обратный вызов, который передает результаты эпохи в файл CSV.
# csv_logger = CSVLogger('training.log')
# csv_logger = [csv_logger]

In [34]:
# Обучаем
history = model_2.fit(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
        # callbacks=csv_logger
        # callbacks = callbacks_stop
)

In [35]:
# Сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model):
model_2.save('../working/model_last.hdf5')
model_2.load_weights('best_model.hdf5')

In [36]:
scores = model_2.evaluate(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [37]:
# Посмотрим на графики обучения:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc)) 
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

## **Предобученная нейронная сеть EfficientNetB2**:

In [38]:
base_model_3 = EfficientNetB2(include_top=False, weights='imagenet', input_tensor=None, 
    pooling=None, classes=1000, classifier_activation='softmax')

In [39]:
# Устанавливаем новую «голову» (head):
x = base_model_3.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(256, activation='relu')(x)
x = Dropout(0.25)(x)
# and a logistic layer -- let's say we have 10 classes
predictions = Dense(CLASS_NUM, activation='softmax')(x)

# this is the model we will train
model_3 = Model(inputs=base_model_3.input, outputs=predictions)
model_3.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=LR), metrics=["accuracy"])

In [40]:
model_3.summary()

In [41]:
# Обучение модели

# Добавим ModelCheckpoint. Эта функция позволяет сохранять прогресс обучения модели, чтобы в нужный момент можно было его подгрузить
# и дообучить модель:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
callbacks_list = [checkpoint]

callbacks_stop = EarlyStopping(patience=5, restore_best_weights=True)
    
# Обратный вызов, который передает результаты эпохи в файл CSV.
# csv_logger = CSVLogger('training.log')
# csv_logger = [csv_logger]

In [42]:
# Обучаем
history = model_3.fit(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
        # callbacks=csv_logger
        # callbacks = callbacks_stop
)

In [43]:
# Сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model):
model_3.save('../working/model_last.hdf5')
model_3.load_weights('best_model.hdf5')

In [44]:
scores = model_3.evaluate(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [45]:
# Посмотрим на графики обучения:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc)) 
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

## **Последовательная архитектура нейронной сети с примененной BatchNormalization:**

In [46]:
base_model_4 = Sequential()
base_model_4.add(Dense(224, input_shape=(input_shape)))
base_model_4.add(Activation('relu'))
base_model_4.add(Conv2D(224, (3, 3)))
base_model_4.add(Activation('relu'))
base_model_4.add(Dense(64))
base_model_4.add(Activation('relu'))
base_model_4.add(Dropout(0.5))
base_model_4.add(Dense(10))
base_model_4.add(Activation('sigmoid'))
base_model_4.add(BatchNormalization())

In [ ]:
# Устанавливаем новую «голову» (head):
x = base_model_4.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(256, activation='relu')(x)
x = Dropout(0.25)(x)
# and a logistic layer -- let's say we have 10 classes
predictions = Dense(CLASS_NUM, activation='softmax')(x)

# this is the model we will train
model_4 = Model(inputs=base_model_4.input, outputs=predictions)
model_4.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=LR), metrics=["accuracy"])

In [ ]:
model_4.summary()

In [ ]:
# Обучение модели

# Добавим ModelCheckpoint. Эта функция позволяет сохранять прогресс обучения модели, чтобы в нужный момент можно было его подгрузить
# и дообучить модель:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
callbacks_list = [checkpoint]

callbacks_stop = EarlyStopping(patience=5, restore_best_weights=True)
    
# Обратный вызов, который передает результаты эпохи в файл CSV.
# csv_logger = CSVLogger('training.log')
# csv_logger = [csv_logger]

In [ ]:
# Обучаем
history = model_4.fit(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
        # callbacks=csv_logger
        # callbacks = callbacks_stop
)

In [ ]:
# Сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model):
model_4.save('../working/model_last.hdf5')
model_4.load_weights('best_model.hdf5')

In [ ]:
scores = model_4.evaluate(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# Посмотрим на графики обучения:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc)) 
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()